In [1]:
import sqlite3
from pxr import Usd, UsdGeom, Gf, Vt, Sdf
import pyvista as pv

In [2]:
def load_table(db_file, table, cn):
   conn = sqlite3.connect(db_file)
   conn.row_factory = sqlite3.Row
   cursor = conn.cursor()
   query = f'SELECT * FROM {table};'
   cursor.execute(query)
   results = cursor.fetchall()
   columns=results[0].keys()
   print(table,len(results), columns)
   res={}
   for r in results:      
      ins={}
      for c in columns[1:]:
         ins[c]=r[c]
      res[r['id']]=type(cn,(),ins)
   cursor.close()
   conn.close()
   return res

In [3]:
database_file = "geometry_atlas.db"
sh_d = load_table(database_file, 'Shapes','SH')

Shapes 119487 ['id', 'type', 'parameters']


In [4]:
def create_box(id, pars):
    dx=pars['XHalfLength']
    dy=pars['YHalfLength']
    dz=pars['ZHalfLength']
    stage: Usd.Stage = Usd.Stage.CreateNew(f"assets/box_{id}.usda")

    box = UsdGeom.Mesh.Define(stage, f'/box_{id}')
    box.CreatePointsAttr([
        (-dx, -dy, -dz), 
        ( dx, -dy, -dz), 
        ( dx,  dy, -dz), 
        (-dx,  dy, -dz),        
        (-dx, -dy,  dz), 
        ( dx, -dy,  dz), 
        ( dx,  dy,  dz), 
        (-dx,  dy,  dz),
        ])
    box.CreateFaceVertexCountsAttr([4,4,4,4,4,4])
    box.CreateFaceVertexIndicesAttr([0, 3, 2, 1, 4, 5, 6, 7, 0, 1, 5, 4, 1, 2, 6, 5, 2, 3, 7, 6, 3, 0, 4, 7])
    box.CreateExtentAttr([(-dx, -dy, -dz), (dx, dy, dz)])
    box.CreateDisplayColorAttr([(0,0,1)])
    stage.Save()

In [5]:
def create_tube(id, pars):
    # Tube {'RMin': 28.995, 'RMax': 245.0, 'ZHalfLength': 3460.0}
    rmin=pars['RMin']
    rmax=pars['RMax']
    dz=pars['ZHalfLength']
    stage: Usd.Stage = Usd.Stage.CreateNew(f"assets/tube_{id}.usda")

    tube = UsdGeom.Mesh.Define(stage, f'/tube_{id}')
    cout = pv.Cylinder(
        center=(0,0,0), direction=(1,0,0), 
        radius=rmax,height=dz*2, 
        resolution=20, capping=True
        ).triangulate(inplace=True)
    cin = pv.Cylinder(radius=rmin, height=dz*2, capping =True).triangulate(inplace=True)
    c = cout.boolean_difference(cin)
    print(c.faces)
    print(c.points)
    # tube.CreatePointsAttr([
    #     (-dx, -dy, -dz), 
    #     ( dx, -dy, -dz), 
    #     ( dx,  dy, -dz), 
    #     (-dx,  dy, -dz),        
    #     (-dx, -dy,  dz), 
    #     ( dx, -dy,  dz), 
    #     ( dx,  dy,  dz), 
    #     (-dx,  dy,  dz),
    #     ])
    # tube.CreateFaceVertexCountsAttr([4,4,4,4,4,4])
    # tube.CreateFaceVertexIndicesAttr([0, 3, 2, 1, 4, 5, 6, 7, 0, 1, 5, 4, 1, 2, 6, 5, 2, 3, 7, 6, 3, 0, 4, 7])
    # tube.CreateExtentAttr([(-dx, -dy, -dz), (dx, dy, dz)])
    tube.CreateDisplayColorAttr([(0,0,1)])
    stage.Save()

In [6]:
n=0
for i,s in sh_d.items():
    if s.type=='UnidentifiedShape':
        print('skipping:', i, s.type, s.parameters)
        continue

    pars=s.parameters.split(';')
    par_d={}
    for p in pars:
        par_d[p.split('=')[0]]=float(p.split('=')[1])
    print(i, s.type, par_d)

    if s.type=='Box': 
        create_box(i, par_d)
    elif s.type=='Tube':
        create_tube(i, par_d)
    else:
        pass
    if i>25: break

1 Box {'XHalfLength': 10000.0, 'YHalfLength': 10000.0, 'ZHalfLength': 10000.0}
2 Tube {'RMin': 28.995, 'RMax': 245.0, 'ZHalfLength': 3460.0}
[  3   0   1 ... 343 345 341]
[[ 3.4600000e+03 -2.4500000e+02  0.0000000e+00]
 [-3.4600000e+03 -2.4500000e+02  0.0000000e+00]
 [ 3.4600000e+03 -2.3300885e+02 -7.5709160e+01]
 ...
 [-3.4600000e+03 -2.8766365e+01  3.6340370e+00]
 [ 3.4600000e+03 -2.8937784e+01  1.8206111e+00]
 [-3.4600000e+03 -2.8937784e+01  1.8206111e+00]]
3 Tube {'RMin': 28.997, 'RMax': 143.0, 'ZHalfLength': 460.0}
[  3   0   1 ... 363 359 361]
[[ 460.        -143.           0.       ]
 [-460.        -143.           0.       ]
 [ 460.        -136.00108    -44.18943  ]
 ...
 [-460.         -28.76835      3.6342878]
 [ 460.         -28.939781     1.8207366]
 [-460.         -28.939781     1.8207366]]
4 Tube {'RMin': 29.757332898676882, 'RMax': 42.001, 'ZHalfLength': 339.101}
[  3   0   1 ... 399 394 397]
[[ 339.101      -42.001        0.       ]
 [-339.101      -42.001        0.     